This walkthrough-style Jupyter notebook will detail the methodology used to evaluate the predictive performance of the underlying forecasting models in the Residential Energy Demand Forecasting Pipeline.

In [67]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import pickle as pkl
import torch as t
from scipy.stats import wilcoxon
import plotly.graph_objects as go
import os
import warnings
import model_definitions as md
import preprocessing as pp
import numpy as np
import get_data as gd
import seaborn as sns
import matplotlib.pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
saved_data_directory = "Saved"
load_model = True

Fit Forecasting Model on all training data

Load best fitting Forecaster object or fit a new one

In [93]:
model_path = r"{}/Models/forecaster.pkl".format(saved_data_directory)

if load_model and os.path.exists(model_path): # load existing model
    with open(model_path, "rb") as file:
        model = pkl.load(file)

else: # fit new model - want to load these from repository
    best_pv_hyperparameters = {'changepoint_prior_scale': 0.001, 'seasonality_prior_scale': 0.01, 'point_var_lags': 10, 'minimum_error_prediction': 100, 'error_trend': 100}
    best_lstm_hyperparameters = {"batch_size":100, "lr":0.0005, "dropout":0.1, "num_layers":1, "hidden_size":32, "sequence_length":24*7*3, "max_epochs":100}

    # load in clean dataset
    clean_data = pd.read_csv(r"Saved/Datasets/clean_training.csv", index_col=0)
    clean_data.index = pd.to_datetime(clean_data.index)
    clean_data.loc[clean_data["HourlyPrecipitation"].isna(),"HourlyPrecipitation"] = "None"
    display(clean_data.shape)
    # clean_data.loc[:,"HourlyPrecipitation"] = clean_data["HourlyPrecipitation"].replace({np.nan:"None"})

    # define model
    model = md.Forecaster(short_term_horizon=24)

    # fit model
    warnings.filterwarnings('ignore')
    model.fit(clean_training_data=clean_data, dependent_variable="Energy Demand (MWH)", 
        hyperparameters=best_pv_hyperparameters | best_lstm_hyperparameters, verbose=False)

(46068, 11)

<br>

# Validation

### Evaluating the Model on Holdout Data

With a final optimized forecasting model defined, it is important to evaluate its performance on unseen data and to contextualize that performance by comparing it with the performance of a baseline model. Below, we load the holdout dataset that was set aside during the processing phase of the pipeline and use it to evaluate the final forecasting model. The *Forecaster* class produces forecasts when the *predict* function is called and given a number, *hours_ahead*, of forecasts to predict. It returns both point forecasts and variance forecasts which can be used to create confidence intervals or relative confidence measures for each point forecasts. 

Below, we produce these forecasts and plot them alongside the actual values. The variance forecasts are used to calculate a 95% confidence interval (CI) for each point forecast.

### Evaluate Long-term Forecasting Performance

In [94]:
# load holdout data
evaluation_data = pd.read_csv(r"Saved/Datasets/holdout.csv", index_col=0)
evaluation_data.index = pd.to_datetime(evaluation_data.index)

# process holdout data
processor_path = r"{}/Models/processor.pkl".format(saved_data_directory)

# load existing processor (this is needed to process evaluation data)
with open(processor_path, "rb") as file:
    processor = pkl.load(file)
    processor.produce_eda_plots = False
    
# processor = pp.PreprocessingPipeline(save_datasets=False, produce_eda_plots=False)
evaluation_data, _ = processor.process_dataset(evaluation_data, train=False) # HERE need to use previous Processor to process this dataset, use the same Prophet models to impute values
evaluation_data.loc[evaluation_data["HourlyPrecipitation"].isna(),"HourlyPrecipitation"] = "None"
display(evaluation_data["HourlyPrecipitation"].value_counts(dropna=False))

Interpolating for variable Energy Demand (MWH)
Interpolating for variable HourlyDryBulbTemperature
Interpolating for variable HourlyDewPointTemperature
Interpolating for variable HourlyWetBulbTemperature
Interpolating for variable HourlyStationPressure
Interpolating for variable HourlyWindSpeed
Interpolating for variable Energy Price (cents/KWH)
Interpolating for variable Civilian Noninstitutional Population
Interpolating for variable Labor Force Participation
Interpolating for variable CPI-U
Interpolating for variable HourlyPrecipitation


HourlyPrecipitation
None           4574
Light Rain      351
Medium Rain     179
Heavy Rain       15
Name: count, dtype: int64

In [95]:
display(evaluation_data[evaluation_data["HourlyPrecipitation"].isna()])

,Energy Demand (MWH),HourlyDryBulbTemperature,HourlyDewPointTemperature,HourlyWetBulbTemperature,HourlyStationPressure,HourlyPrecipitation,HourlyWindSpeed,Energy Price (cents/KWH),Civilian Noninstitutional Population,Labor Force Participation,CPI-U


In [96]:
# get complete range to be predicted (in case the holdout set is missing any values)
complete_range = pd.date_range(start=evaluation_data.index.min(), end=evaluation_data.index.max(), freq='H')

# produce long-term forecasts
lt_forecasts = model.long_term_predict(complete_range.shape[0])

# reconcile forecasts and ground truth values
results = pd.merge(lt_forecasts, evaluation_data, left_index=True, right_index=True, how="inner")
point_forecasts = results["Point Forecasts"].values
variance_forecasts = results["Variance Forecasts"].values
actual_values = results["Energy Demand (MWH)"]

display(lt_forecasts)

,Point Forecasts,Variance Forecasts
ds,,
2023-09-20 17:00:00,5939.596219,261792.939704
2023-09-20 18:00:00,5994.287436,289862.793621
2023-09-20 19:00:00,6048.001362,326691.119936
2023-09-20 20:00:00,6090.931726,364605.382037
2023-09-20 21:00:00,6112.973559,388089.972322
...,...,...
2024-04-20 20:00:00,4851.548245,679031.641875
2024-04-20 21:00:00,4846.972954,700218.746008
2024-04-20 22:00:00,4820.972776,694866.752498


In [97]:
# Combine error forecasts
std_forecasts = np.sqrt(variance_forecasts)

# 1.96 is the z-value for the 2.5th percentile of the standard normal distribution
lower_cuts = point_forecasts - 1.96 * std_forecasts  
upper_cuts = point_forecasts + 1.96 * std_forecasts

# Create figure showing forecasts and actual values
long_term_fig = go.Figure()
long_term_fig.add_trace(go.Scatter(x=evaluation_data.index, y=point_forecasts, mode='lines', 
    name='Point Forecasts', line=dict(color='rgba(255, 0, 0)')))
long_term_fig.add_trace(go.Scatter(x=evaluation_data.index, y=actual_values, mode='lines', 
    name='Ground Truth', line=dict(color='rgba(0, 0, 255)')))
long_term_fig.add_trace(go.Scatter(x=evaluation_data.index, y=upper_cuts, mode='lines', 
    name='Upper 95 CI', line=dict(color='rgba(0, 255, 0, 0.2)')))
long_term_fig.add_trace(go.Scatter(x=evaluation_data.index, y=lower_cuts, mode='lines', 
    name='Lower 95 CI', line=dict(color='rgba(200, 125, 200, 0.2)')))

# Customize the layout
long_term_fig.update_layout(
    title="Long-term Evaluation of Forecasting Pipeline with Holdout Data",
    xaxis_title='Time',
    yaxis_title='Energy Demand (MWH)',
    template='plotly_dark' 
)

# save plotly figure for web application
with open(r"assets/long_term_evaluation_plot.pkl", 'wb') as file:
    pkl.dump(long_term_fig, file=file)

long_term_fig.show()

In [98]:
### statistical comparison between baseline forecasting model and created forecasting model ###

# define baseline model as simple year-long moving average
sma = clean_data["Energy Demand (MWH)"].rolling(window=24*365).mean()
baseline_prediction = sma.iloc[-1]
sma_error = pd.merge(clean_data["Energy Demand (MWH)"], sma, left_index=True, right_index=True, 
    how="inner", suffixes=["_actual", "_baseline_prediction"])
sma_error = sma_error[sma_error["Energy Demand (MWH)_baseline_prediction"].notna()]
sma_variance = pd.Series((sma_error["Energy Demand (MWH)_baseline_prediction"] - 
    sma_error["Energy Demand (MWH)_actual"]).values**2).rolling(window=24*365).mean()
baseline_variance = sma_variance.iloc[-1]

# calculate errors from baseline model
baseline_errors = actual_values - baseline_prediction

# calculate errors from project model
model_errors = actual_values - point_forecasts

### calculate all measures for Forecasting Pipeline's long-term forecasts ###
# mse
pipeline_mse = np.sum((model_errors**2))/model_errors.shape[0]

# wmse
relative_confidence = 1/std_forecasts
relative_confidence = relative_confidence/np.sum(relative_confidence) * relative_confidence.shape[0]
pipeline_wmse = np.sum(relative_confidence * 
    (point_forecasts - actual_values)**2)/actual_values.shape[0]

# mae
pipeline_mae = np.sum(np.abs(model_errors))/model_errors.shape[0]

# mape
pipeline_mape = np.sum(np.abs(model_errors/actual_values))/model_errors.shape[0]

### calculate all measures for baseline model ###
# mse
baseline_mse = np.sum((baseline_errors**2))/baseline_errors.shape[0]
baseline_wmse = baseline_mse

# mae
baseline_mae = np.sum(np.abs(baseline_errors))/baseline_errors.shape[0]

# mape
baseline_mape = np.sum(np.abs(baseline_errors/actual_values))/baseline_errors.shape[0]

# calculate all measures for EIA forecasts

# compare MSE values
print("Baseline MSE: {:.2f}".format(baseline_mse))
print("Project Model MSE: {:.2f}".format(pipeline_mse))

# Perform Wilcoxon signed-rank test
statistic, p_value = wilcoxon(baseline_errors, model_errors)

# Output the test statistic and p-value
print("\nWilcoxon signed-rank test statistic:", statistic)
print("p-value: {:.4f}".format(p_value))

# Interpret the results
print("Outcome of Statistical Test:")
if p_value < 0.05:
    print("Reject the null hypothesis. There is a significant difference between the two models.")
else:
    print("Fail to reject the null hypothesis. There is no significant difference between the two models.")

# compare weighted MSE to regular MSE to see if confidence values help

print("\nProject Model Weighted MSE: {:.2f}".format(pipeline_wmse))

Baseline MSE: 601707.48
Project Model MSE: 131132.72

Wilcoxon signed-rank test statistic: 2905955.0
p-value: 0.0000
Outcome of Statistical Test:
Reject the null hypothesis. There is a significant difference between the two models.

Project Model Weighted MSE: 128252.87


In [99]:
# include MSE, wmse, Mape, weighted mape, statistical significance of each compared to baseline model
# create dataframe with results - easy to convert to html.
long_results_df = pd.DataFrame(index=["Forecasting Pipeline", "Baseline Yearly MA"], 
    data={"MSE":[pipeline_mse, baseline_mse], "Weighted MSE":[pipeline_wmse, baseline_wmse], 
    "MAE":[pipeline_mae, baseline_mae], "MAPE":[pipeline_mape, baseline_mape], "Wilcoxon Test p-value":[p_value, np.nan]}).round(2)
long_results_df.to_csv(r"assets/long_term_performance_report.csv")

### Evaluate Short-term Forecasting Performance

In [20]:
eia_forecast_data = gd.get_eia_forecasts(start=np.datetime64("2018-06-19"), end=np.datetime64("2024-04-21"))["EIA Forecast (MWH)"]

Requesting EIA energy demand forecast data from EIA over 11 requests


Obtain baseline model from EIA

In [21]:
eia_forecasts = eia_forecast_data[eia_forecast_data.index < evaluation_data.index.min()]
training_energy_demand = pd.read_csv(r"Saved/Datasets/preliminary.csv", index_col=0)["Energy Demand (MWH)"]
training_energy_demand.index = pd.to_datetime(training_energy_demand.index)
training_energy_demand = training_energy_demand[(training_energy_demand.index < evaluation_data.index.min())]
eia_forecasts = eia_forecasts[eia_forecasts.index.isin(training_energy_demand.index)]

coef = np.dot(eia_forecasts.values, training_energy_demand.values)/np.dot(eia_forecasts.values, eia_forecasts.values)
eia_forecasts = eia_forecast_data[eia_forecast_data.index.isin(evaluation_data.index)] * coef
display(eia_forecasts.shape)

(5095,)

In [13]:
sequence_length = 3*7*24
last_n_training = clean_data.iloc[-(sequence_length+model.short_term_horizon):]
evaluation_data_st = pd.concat([last_n_training, evaluation_data], axis=0)
evaluation_data_st.loc[evaluation_data_st["HourlyPrecipitation"].isna(),"HourlyPrecipitation"] = "None"

Calculate baseline short-term moving average

In [48]:
# define baseline short-term forecasting model as simple daily moving average
short_term_ma_period = model.short_term_horizon
last_n_training = clean_data.iloc[-(short_term_ma_period*2):]
evaluation_data_st = pd.concat([last_n_training, evaluation_data], axis=0)
evaluation_data_st.loc[evaluation_data_st["HourlyPrecipitation"].isna(),"HourlyPrecipitation"] = "None"

sma = evaluation_data_st["Energy Demand (MWH)"].rolling(window=short_term_ma_period).mean()
# sma = evaluation_data_st["Energy Demand (MWH)"].shift(short_term_ma_period)
sma = sma[sma.notna()]
sma_error = pd.merge(evaluation_data_st["Energy Demand (MWH)"], sma, left_index=True, right_index=True, 
    how="inner", suffixes=["_actual", "_baseline_prediction"])
sma_variance = pd.Series((sma_error["Energy Demand (MWH)_baseline_prediction"] - 
    sma_error["Energy Demand (MWH)_actual"]).values**2).rolling(window=short_term_ma_period).mean()

sma_forecasts = pd.DataFrame(index=sma.index, data={"Baseline MA Point Forecast":sma.values, "Baseline MA Error Forecast":sma_variance.values})
sma_forecasts = sma_forecasts[sma_forecasts["Baseline MA Error Forecast"].notna()]

Calculate Forecasting Pipeline short-term forecasts

In [100]:
sequence_length = 3*7*24
last_n_training = clean_data.iloc[-(sequence_length+model.short_term_horizon):]
evaluation_data_st = pd.concat([last_n_training, evaluation_data], axis=0)
evaluation_data_st.loc[evaluation_data_st["HourlyPrecipitation"].isna(),"HourlyPrecipitation"] = "None"

st_forecasts = model.short_term_predict(evaluation_data_st, expected_output_size=evaluation_data.shape[0]-model.short_term_horizon-sequence_length, 
    lstm_sequence_length=sequence_length)
display(st_forecasts)

['Energy Demand (MWH)', 'HourlyDryBulbTemperature', 'HourlyDewPointTemperature', 'HourlyWetBulbTemperature', 'HourlyStationPressure', 'HourlyWindSpeed', 'Energy Price (cents/KWH)', 'Civilian Noninstitutional Population', 'Labor Force Participation', 'CPI-U', 'HourlyPrecipitation_Light Rain', 'HourlyPrecipitation_Medium Rain', 'HourlyPrecipitation_None']
['Energy Demand (MWH)', 'HourlyDryBulbTemperature', 'HourlyDewPointTemperature', 'HourlyWetBulbTemperature', 'HourlyStationPressure', 'HourlyWindSpeed', 'Energy Price (cents/KWH)', 'Civilian Noninstitutional Population', 'Labor Force Participation', 'CPI-U', 'HourlyPrecipitation_Light Rain', 'HourlyPrecipitation_Medium Rain', 'HourlyPrecipitation_None']


,Point Forecast,Variance Forecast
2023-09-20 17:00:00,5919.710449,3607.277532
2023-09-20 18:00:00,5974.167969,3606.781055
2023-09-20 19:00:00,6016.044922,3607.199803
2023-09-20 20:00:00,6054.579102,3606.186030
2023-09-20 21:00:00,6051.221680,3607.495422
...,...,...
2024-04-20 20:00:00,4887.214844,3606.934263
2024-04-20 21:00:00,4883.117676,3607.005887
2024-04-20 22:00:00,4958.108398,3607.091146
2024-04-20 23:00:00,5010.516113,3606.618660


In [101]:
st_forecast_data = pd.concat([st_forecasts, eia_forecasts, sma_forecasts, actual_values], axis=1)
st_forecast_data = st_forecast_data[st_forecast_data.notna().all(axis=1)]

display(st_forecast_data)
st_forecasts = st_forecast_data[["Point Forecast", "Variance Forecast"]]
eia_forecasts = st_forecast_data["EIA Forecast (MWH)"]
ma_forecasts = st_forecast_data[["Baseline MA Point Forecast", "Baseline MA Error Forecast"]]
st_actual_values = st_forecast_data["Energy Demand (MWH)"]

,Point Forecast,Variance Forecast,EIA Forecast (MWH),Baseline MA Point Forecast,Baseline MA Error Forecast,Energy Demand (MWH)
2023-09-20 17:00:00,5919.710449,3607.277532,5072.488843,5353.958333,559864.600622,5981.0
2023-09-20 18:00:00,5974.167969,3606.781055,5159.748807,5351.791667,563665.668113,6073.0
2023-09-20 19:00:00,6016.044922,3607.199803,5221.502012,5351.291667,569496.379702,6153.0
2023-09-20 20:00:00,6054.579102,3606.186030,5345.679654,5351.791667,576703.492332,6242.0
2023-09-20 21:00:00,6051.221680,3607.495422,5546.377571,5354.166667,587358.720269,6361.0
...,...,...,...,...,...,...
2024-04-20 20:00:00,4887.214844,3606.934263,4248.217796,4618.166667,279332.026042,4630.0
2024-04-20 21:00:00,4883.117676,3607.005887,4366.354363,4588.166667,271608.113932,4667.0
2024-04-20 22:00:00,4958.108398,3607.091146,4592.223424,4558.750000,263231.402344,4705.0
2024-04-20 23:00:00,5010.516113,3606.618660,4810.708950,4534.208333,258075.901765,4757.0


In [102]:
# Create figure showing forecasts and actual values
short_term_fig = go.Figure()
short_term_fig.add_trace(go.Scatter(x=st_actual_values.index, y=eia_forecasts, mode='lines', 
    name='EIA Day-Ahead Forecasts', line=dict(color='rgba(255, 0, 0)')))
short_term_fig.add_trace(go.Scatter(x=st_actual_values.index, y=st_actual_values, mode='lines', 
    name='Ground Truth', line=dict(color='rgba(0, 0, 255)')))
short_term_fig.add_trace(go.Scatter(x=st_actual_values.index, y=st_forecasts["Point Forecast"], mode='lines', 
    name='Predicted Forecasts', line=dict(color='rgba(0, 0, 255)')))
short_term_fig.add_trace(go.Scatter(x=st_actual_values.index, y=ma_forecasts["Baseline MA Point Forecast"], mode='lines', 
    name='Baseline Moving Average', line=dict(color='rgba(0, 0, 255)')))

# Customize the layout
short_term_fig.update_layout(
    title="Short-term Evaluation of Forecasting Pipeline with Holdout Data",
    xaxis_title='Time',
    yaxis_title='Energy Demand (MWH)',
    template='plotly_dark' 
)
short_term_fig.show()

with open(r"assets/short_term_evaluation_plot.pkl", "wb") as file:
    pkl.dump(short_term_fig, file)

In [103]:
### statistical comparison between Pipeline Forecasting Model and EIA (short-term forecasts) ###

# calculate errors from baseline model
baseline_errors = st_actual_values - ma_forecasts["Baseline MA Point Forecast"]

# calculate errors from project model
model_errors = st_actual_values - st_forecasts["Point Forecast"].values

### calculate all measures for Forecasting Pipeline's short-term forecasts ###
# mse
pipeline_mse = np.sum((model_errors**2))/model_errors.shape[0]

# wmse
relative_confidence = 1/st_forecasts["Variance Forecast"].values
relative_confidence = relative_confidence/np.sum(relative_confidence) * relative_confidence.shape[0]
pipeline_wmse = np.sum(relative_confidence * 
    (st_forecasts["Point Forecast"] - st_actual_values)**2)/st_actual_values.shape[0]

# mae
pipeline_mae = np.sum(np.abs(model_errors))/model_errors.shape[0]

# mape
pipeline_mape = np.sum(np.abs(model_errors/st_actual_values))/model_errors.shape[0]

### calculate all measures for baseline model ###
# mse
baseline_mse = np.sum((baseline_errors**2))/baseline_errors.shape[0]
baseline_wmse = baseline_mse

# mae
baseline_mae = np.sum(np.abs(baseline_errors))/baseline_errors.shape[0]

# mape
baseline_mape = np.sum(np.abs(baseline_errors/st_actual_values))/baseline_errors.shape[0]

# calculate all measures for EIA forecasts

# compare MSE values
print("Baseline MSE: {:.2f}".format(baseline_mse))
print("Project Model MSE: {:.2f}".format(pipeline_mse))

# Perform Wilcoxon signed-rank test
statistic, p_value = wilcoxon(baseline_errors, model_errors)

# Output the test statistic and p-value
print("\nWilcoxon signed-rank test statistic:", statistic)
print("p-value: {:.4f}".format(p_value))

# Interpret the results
print("Outcome of Statistical Test:")
if p_value < 0.05:
    print("Reject the null hypothesis. There is a significant difference between the two models.")
else:
    print("Fail to reject the null hypothesis. There is no significant difference between the two models.")

# compare weighted MSE to regular MSE to see if confidence values help

print("\nProject Model Weighted MSE: {:.2f}".format(pipeline_wmse))

Baseline MSE: 357804.17
Project Model MSE: 170066.59

Wilcoxon signed-rank test statistic: 5855680.0
p-value: 0.0000
Outcome of Statistical Test:
Reject the null hypothesis. There is a significant difference between the two models.

Project Model Weighted MSE: 170061.86


In [104]:
### statistical comparison between Pipeline Forecasting Model and EIA (short-term forecasts) ###

# calculate errors from baseline model
eia_errors = st_actual_values - eia_forecasts

# calculate errors from project model
model_errors = st_actual_values - st_forecasts["Point Forecast"]

### calculate all measures for Forecasting Pipeline's short-term forecasts ###
# mse
pipeline_mse = np.sum((model_errors**2))/model_errors.shape[0]

# wmse
relative_confidence = 1/st_forecasts["Variance Forecast"].values
relative_confidence = relative_confidence/np.sum(relative_confidence) * relative_confidence.shape[0]
pipeline_wmse = np.sum(relative_confidence * 
    (st_forecasts["Point Forecast"] - st_actual_values)**2)/st_actual_values.shape[0]

# mae
pipeline_mae = np.sum(np.abs(model_errors))/model_errors.shape[0]

# mape
pipeline_mape = np.sum(np.abs(model_errors/st_actual_values))/model_errors.shape[0]

### calculate all measures for baseline model ###
# mse
eia_mse = np.sum((eia_errors**2))/eia_errors.shape[0]
eia_wmse = baseline_mse

# mae
eia_mae = np.sum(np.abs(eia_errors))/eia_errors.shape[0]

# mape
eia_mape = np.sum(np.abs(eia_errors/st_actual_values))/eia_errors.shape[0]

# calculate all measures for EIA forecasts

# compare MSE values
print("Baseline MSE: {:.2f}".format(eia_mse))
print("Project Model MSE: {:.2f}".format(pipeline_mse))

# Perform Wilcoxon signed-rank test
eia_statistic, eia_p_value = wilcoxon(eia_errors, model_errors)

# Output the test statistic and p-value
print("\nWilcoxon signed-rank test statistic:", eia_statistic)
print("p-value: {:.4f}".format(eia_p_value))

# Interpret the results
print("Outcome of Statistical Test:")
if eia_p_value < 0.05:
    print("Reject the null hypothesis. There is a significant difference between the two models.")
else:
    print("Fail to reject the null hypothesis. There is no significant difference between the two models.")

# compare weighted MSE to regular MSE to see if confidence values help

print("\nProject Model Weighted MSE: {:.2f}".format(pipeline_wmse))

Baseline MSE: 110498.24
Project Model MSE: 170066.59

Wilcoxon signed-rank test statistic: 3178745.0
p-value: 0.0000
Outcome of Statistical Test:
Reject the null hypothesis. There is a significant difference between the two models.

Project Model Weighted MSE: 170061.86


In [105]:
# include MSE, wmse, Mape, weighted mape, statistical significance of each compared to baseline model
# create dataframe with results - easy to convert to html.
day_ahead_results_df = pd.DataFrame(index=["Forecasting Pipeline", "Baseline Moving Avg", "EIA Forecasts"], 
    data={"MSE":[pipeline_mse, baseline_mse, eia_mse], "Weighted MSE":[pipeline_wmse, baseline_wmse, np.nan], 
    "MAE":[pipeline_mae, baseline_mae, eia_mae], "MAPE":[pipeline_mape, baseline_mape, eia_mape], "Wilcoxon Test p-value":[np.nan, p_value, eia_p_value]})
day_ahead_results_df = day_ahead_results_df.replace({"TBD":-888}).astype(float).round(2).replace({-888:"TBD"})
display(day_ahead_results_df)
day_ahead_results_df.to_csv(r"assets/short_term_performance_report.csv")

,MSE,Weighted MSE,MAE,MAPE,Wilcoxon Test p-value
Forecasting Pipeline,170066.59,170061.86,316.14,0.06,NaN
Baseline Moving Avg,357804.17,357804.17,518.69,0.10,0.0
EIA Forecasts,110498.24,NaN,286.98,0.06,0.0


### Conduct Sensitivity Analysis

In [73]:
display(evaluation_data_st.columns)

Index(['Energy Demand (MWH)', 'HourlyDryBulbTemperature',
       'HourlyDewPointTemperature', 'HourlyWetBulbTemperature',
       'HourlyStationPressure', 'HourlyPrecipitation', 'HourlyWindSpeed',
       'Energy Price (cents/KWH)', 'Civilian Noninstitutional Population',
       'Labor Force Participation', 'CPI-U'],
      dtype='object')

In [110]:
display(evaluation_data)
display(clean_data)

,Energy Demand (MWH),HourlyDryBulbTemperature,HourlyDewPointTemperature,HourlyWetBulbTemperature,HourlyStationPressure,HourlyPrecipitation,HourlyWindSpeed,Energy Price (cents/KWH),Civilian Noninstitutional Population,Labor Force Participation,CPI-U
2023-09-20 17:00:00,5981.0,71.000000,48.000000,58.000000,29.970000,None,3.000000,23.330000,6.744964e+06,61.200000,325.613000
2023-09-20 18:00:00,6073.0,70.000000,49.000000,58.000000,29.990000,None,0.000000,23.330000,6.744964e+06,61.200000,325.613000
2023-09-20 19:00:00,6153.0,67.000000,50.000000,58.000000,30.010000,None,0.000000,23.330000,6.744964e+06,61.200000,325.613000
2023-09-20 20:00:00,6242.0,67.000000,50.000000,58.000000,30.020000,None,0.000000,23.330000,6.744964e+06,61.200000,325.613000
2023-09-20 21:00:00,6361.0,67.000000,49.000000,57.000000,30.030000,None,0.000000,23.330000,6.744964e+06,61.200000,325.613000
...,...,...,...,...,...,...,...,...,...,...,...
2024-04-20 20:00:00,4630.0,54.590759,36.010170,46.328358,29.691337,None,4.925700,20.364970,6.724343e+06,62.133938,330.962073
2024-04-20 21:00:00,4667.0,54.002312,36.095989,46.091286,29.694715,None,4.912757,20.365914,6.724350e+06,62.135956,330.960387
2024-04-20 22:00:00,4705.0,53.445620,36.162767,45.840379,29.694745,None,4.940276,20.367155,6.724327e+06,62.138073,330.962845
2024-04-20 23:00:00,4757.0,52.895919,36.218548,45.580553,29.692206,None,4.986535,20.368748,6.724270e+06,62.140308,330.970139


,Energy Demand (MWH),HourlyDryBulbTemperature,HourlyDewPointTemperature,HourlyWetBulbTemperature,HourlyStationPressure,HourlyPrecipitation,HourlyWindSpeed,Energy Price (cents/KWH),Civilian Noninstitutional Population,Labor Force Participation,CPI-U
2018-06-19 05:00:00,7221.0,78.0,72.0,74.0,29.62,None,6.000000,19.28,7105823.0,60.1,274.170
2018-06-19 06:00:00,6911.0,79.0,70.0,73.0,29.63,None,6.000000,19.28,7105823.0,60.1,274.170
2018-06-19 07:00:00,6691.0,81.0,69.0,73.0,29.63,None,3.672242,19.28,7105823.0,60.1,274.170
2018-06-19 08:00:00,6582.0,82.0,67.0,72.0,29.63,None,3.920117,19.28,7105823.0,60.1,274.170
2018-06-19 09:00:00,6600.0,81.0,69.0,73.0,29.65,None,7.000000,19.28,7105823.0,60.1,274.170
...,...,...,...,...,...,...,...,...,...,...,...
2023-09-20 12:00:00,5160.0,72.0,48.0,59.0,29.99,None,7.000000,23.33,6744964.0,61.2,325.613
2023-09-20 13:00:00,5461.0,72.0,47.0,58.0,29.98,None,0.000000,23.33,6744964.0,61.2,325.613
2023-09-20 14:00:00,5649.0,72.0,49.0,59.0,29.97,None,5.000000,23.33,6744964.0,61.2,325.613
2023-09-20 15:00:00,5772.0,72.0,49.0,59.0,29.97,None,4.456486,23.33,6744964.0,61.2,325.613


In [119]:
sequence_length = 3*7*24

mse_diffs = {}
print(round(pipeline_mse, 2))
for variable in clean_data.columns:
    last_n_training = clean_data.iloc[-(sequence_length+model.short_term_horizon):]
    evaluation_data_st = pd.concat([last_n_training, evaluation_data], axis=0)
    evaluation_data_st.loc[evaluation_data_st["HourlyPrecipitation"].isna(), "HourlyPrecipitation"] = "None"
    # display(evaluation_data_st)
    # print(variable)
    # display(np.random.shuffle(evaluation_data_st[variable].values))
    # evaluation_data_st.loc[:,variable] = np.random.shuffle(evaluation_data_st[variable].values)
    evaluation_data_st.loc[:,variable] = evaluation_data_st[variable].sample(frac=1, random_state=0)
    # display(evaluation_data_st.columns)
    # display(evaluation_data_st)

    st_forecasts = model.short_term_predict(evaluation_data_st, expected_output_size=evaluation_data.shape[0]-model.short_term_horizon-sequence_length, 
        lstm_sequence_length=sequence_length)
    
    st_forecast_data = pd.concat([st_forecasts, actual_values], axis=1)
    st_forecast_data = st_forecast_data[st_forecast_data.notna().all(axis=1)]

    model_forecasts = st_forecast_data["Point Forecast"]
    st_actual_values = st_forecast_data["Energy Demand (MWH)"]
    model_errors = st_actual_values - model_forecasts
    new_pipeline_mse = np.sum((model_errors**2))/model_errors.shape[0]
    mse_diffs[variable] = new_pipeline_mse - pipeline_mse
    print(variable, round(new_pipeline_mse, 2))
feature_importances = pd.Series(mse_diffs)

170066.59
Energy Demand (MWH) 169771.42
HourlyDryBulbTemperature 169771.42
HourlyDewPointTemperature 169771.42
HourlyWetBulbTemperature 169771.42
HourlyStationPressure 169771.42
HourlyPrecipitation 169771.42


KeyboardInterrupt: 

In [ ]:
# Sensitivity Analysis for day-ahead forecasts 
sa_fig = plt.figure(figsize=(8,5))
ax = sa_fig.add_subplot()
sns.barplot(x=[0.85, 0.11, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05], y=clean_data.columns)
ax.bar_label(ax.containers[0])
ax.set_ylabel("Variable")
ax.set_xlabel("Feature Importance")
ax.set_title("Forecasting Model Feature Importance (Demonstration Only)")
ax.set_xlim([0,1])
plt.tight_layout()
pp.save_png_encoded(r'assets/sensitivity_analysis.png', sa_fig)

### Create Report for Download on the Web Application

In [ ]:
from plotly.io import to_html
with open(r"assets/model_performance_report.html", "w") as file:
    file.write("""
        <!DOCTYPE html>
        <html>
        <head>
        <title> Energy Demand Forecasting Model Performance Report </title>
        </head>
        <body>
        <H1> Energy Demand Forecasting Model Performance Report </H1>
        <div style="width: 75%;">The following report contains model performance metrics for the NY City Hourly Probabilistic Residential Energy Demand Forecasting Pipeline. 
            Model performance was evaluated on both long-term and day-ahead forecasts. Evaluation was conducted using a holdout dataset of hourly energy 
            demand values between {} and {}.
        <br>
        <H3> Long-Term Forecasting Performance </H3>
        <div style="width: 75%;"> The following table contains performance metrics for the forecasting model compared with a yearly moving average baseline model.</div>
        <div style="width: 75%;">{}</div>
        <br>
        <div style="width: 75%;"> The following Plotly Figure helps to contextualize the forecasting model's performance by showing its predictions along with the actual energy demand 
            values. It also presents the 95% confidence interval bounds estimated by the forecasting model.</div>
        <div style="width: 75%;">{}</div>
        <br> <br> <br>  
        <H3> Day-Ahead Forecasting Performance </H3>
        <div style="width: 75%;"> The following table contains performance metrics for the forecasting model compared with a yearly moving average baseline model.</div>
        <div style="width: 75%;">{}</div>
        <br>
        <divs style="width: 75%;"> The following Plotly Figure helps to contextualize the forecasting model's performance by showing its predictions along with the actual energy demand 
            values. It also presents the 95% confidence interval bounds estimated by the forecasting model. </div>
        <div style="width: 75%;">{}</div>
        <br> <br> <br>
        <H3>Model Sensitivity Analysis</H3>
        <div style="width: 75%;">The following data has been made up for demonstration purposes. Actual data will be provided in a future update.</div>
        <br>
        <img src="{}" width="50%" alt="Sensitivity Analysis">
        </body>
        """.format(evaluation_data.index.min().strftime("%Y-%m-%d"), evaluation_data.index.max().strftime("%Y-%m-%d"), long_results_df.to_html(), 
            to_html(long_term_fig), day_ahead_results_df.to_html(), to_html(short_term_fig), r"sensitivity_analysis.png"))

## Produce Final Model and Predict into the Future

### Fit Model on All Available Data

In [ ]:
# create new processing pipeline to process all data but not create visuals
processor = pp.PreprocessingPipeline(save_datasets=False, produce_eda_plots=False)
final_train_data,_ = processor.process_dataset(preliminary_dataset=preliminary_data, split_into_train_holdout=False)

In [ ]:
# fit final model on all available data
final_model = md.Forecaster()
final_model.fit(clean_training_data=final_train_data, dependent_variable="Energy Demand (MWH)", 
    strong_predictors=["HourlyDryBulbTemperature"], hyperparameters=best_set)

In [ ]:
# Forecast 2 years into the future
future_point_forecasts, future_variance_forecasts = final_model.predict(hours_ahead=365*24*2)

In [ ]:
# produce plot showing 2 year forecast
# Combine error forecasts
std_forecasts = np.sqrt(future_variance_forecasts)

# 1.96 is the z-value for the 2.5th percentile of the standard normal distribution
lower_cuts = future_point_forecasts - 1.96 * std_forecasts  
upper_cuts = future_point_forecasts + 1.96 * std_forecasts

# Create figure showing forecasts and actual values
fig = go.Figure()
fig.add_trace(go.Scatter(x=future_point_forecasts.index, y=future_point_forecasts, mode='lines', 
    name='Point Forecasts', line=dict(color='rgba(255, 0, 0)')))
fig.add_trace(go.Scatter(x=future_point_forecasts.index, y=upper_cuts, mode='lines', 
    name='Upper 95 CI', line=dict(color='rgba(0, 255, 0, 0.2)')))
fig.add_trace(go.Scatter(x=future_point_forecasts.index, y=lower_cuts, mode='lines', 
    name='Lower 95 CI', line=dict(color='rgba(200, 125, 200, 0.2)')))

# Customize the layout
fig.update_layout(
    title="Evaluating Forecasting Pipeline with Holdout Data",
    xaxis_title='Time',
    yaxis_title='Energy Demand (MWH)',
    template='plotly_dark',
    xaxis=dict(range=[future_point_forecasts.index[0],future_point_forecasts.index[-1]])
)

In [ ]:
# save plotly figure for web app
with open(r"Saved/Plotly Figures/Forecasting/future_forecasts.pkl", 'wb') as file:
    pkl.dump(fig, file=file)

# save future forecasts as csv for web app
future_forecasts = pd.concat([future_point_forecasts.rename("Point Forecast"), future_variance_forecasts.rename("Variance Forecast")], 
    axis=1).rename_axis("Time").reset_index()
future_forecasts.to_csv("assets/future_forecasts.csv")